<a href="https://colab.research.google.com/github/rennyatwork/CegepSteFoy_ReinfLearn_Work/blob/main/TP01/taxi_dqn_v03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gymnasium
!pip install segment_tree

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 10.6 MB/s eta 0:00:00


In [ ]:
import random
import numpy as np
import tensorflow as tf
from collections import deque
import matplotlib.pyplot as plt
import gymnasium as gym

# ... (Your existing imports and DQNModel class)

import numpy as np

class SumTree:
    def __init__(self, capacity):
        self.capacity = capacity
        self.tree = np.zeros(2 * capacity - 1)  # Array to store priority sums
        self.data = np.zeros(capacity, dtype=object)  # Array to store experiences
        self.n_entries = 0  # Current number of entries in the tree
        self.write = 0  # Current index for writing new entries

    def _propagate(self, idx, change):
        """Updates priority values up the tree."""
        parent = (idx - 1) // 2
        self.tree[parent] += change
        if parent != 0:  # Recursively update until root
            self._propagate(parent, change)

    def _retrieve(self, idx, s):
        """Finds the leaf node corresponding to priority 's'."""
        left = 2 * idx + 1
        right = left + 1

        if left >= len(self.tree):  # If reached a leaf node
            return idx

        if s <= self.tree[left]:  # If 's' is less than left child's priority
            return self._retrieve(left, s)  # Search in left subtree
        else:
            return self._retrieve(right, s - self.tree[left])  # Search in right subtree

    def total(self):
        """Returns the total priority sum (root of the tree)."""
        return self.tree[0]

    def add(self, priority, data):
        """Adds a new experience with priority to the tree."""
        idx = self.write + self.capacity - 1  # Calculate leaf node index

        self.data[self.write] = data  # Store experience in data array
        self.update(idx, priority)  # Update priority value

        self.write += 1
        if self.write >= self.capacity:  # Wrap around if exceeding capacity
            self.write = 0

        if self.n_entries < self.capacity:  # Increment entries if not full
            self.n_entries += 1

    def update(self, idx, priority):
        """Updates the priority of an existing experience."""
        change = priority - self.tree[idx]  # Calculate the change in priority
        self.tree[idx] = priority  # Update priority in the tree
        self._propagate(idx, change)  # Propagate the change up the tree

    def get(self, s):
        """Gets the experience and priority associated with a priority value 's'."""
        idx = self._retrieve(0, s)  # Find leaf node index
        dataIdx = idx - self.capacity + 1  # Calculate data array index

        return (idx, self.tree[idx], self.data[dataIdx])  # Return index, priority, and experience



class PrioritizedReplayBuffer:
    def __init__(self, capacity, alpha=0.6, beta=0.4, epsilon=0.01):
        self.tree = SumTree(capacity)
        self.alpha = alpha  # Controls how much prioritization is used
        self.beta = beta  # Importance-sampling, from initial value increasing to 1
        self.epsilon = epsilon  # Small positive constant to ensure all experiences have some probability of being sampled
        self.capacity = capacity

    def add(self, experience, error=1.0):
        priority = (error + self.epsilon) ** self.alpha
        self.tree.add(priority, experience)

    def sample(self, batch_size):
        minibatch = []
        indices = []
        priorities = []

        segment_size = self.tree.total() / batch_size
        for i in range(batch_size):
            a = segment_size * i
            b = segment_size * (i + 1)
            s = random.uniform(a, b)
            (idx, priority, data) = self.tree.get(s)
            priorities.append(priority)
            minibatch.append(data)
            indices.append(idx)

        # Importance sampling weights
        sampling_probabilities = priorities / self.tree.total()
        is_weights = np.power(self.tree.n_entries * sampling_probabilities, -self.beta)
        is_weights /= is_weights.max()

        return minibatch, indices, is_weights

    def update(self, idx, error):
        priority = (error + self.epsilon) ** self.alpha
        self.tree.update(idx, priority)

    def __len__(self):
        return self.tree.n_entries

In [ ]:
import gymnasium as gym
import numpy as np
import tensorflow as tf
from collections import deque
import random
import matplotlib.pyplot as plt

# Environment Setup
env = gym.make('Taxi-v3', render_mode="rgb_array")
state_size = env.observation_space.n  # Taxi-v3 has 500 states
action_size = env.action_space.n

# DQN Model
class DQNModel:
    def __init__(self, state_size, action_size):
        self.model = self._build_model(state_size, action_size)
        self.target_model = tf.keras.models.clone_model(self.model)
        self.target_model.set_weights(self.model.get_weights())

    def _build_model(self, state_size, action_size):
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(64, activation='relu', input_shape=(state_size,)),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(action_size, activation='linear')
        ])
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                      loss='mse')
        return model

    def predict(self, state):
        print('[BEGIN predict]')
        #print(f'[state] : {state} ')
        # Ensure state is passed as a numpy array with the correct dimensions
        if np.isscalar(state):
            state = np.eye(state_size)[state]  # Convert scalar to one-hot encoded array
        else:
            assert state.shape == (state_size,), f"Input shape mismatch, got {state.shape}, expected {(state_size,)}"
        print('[END predict]')
        return self.model.predict(np.array([state]), verbose = 0)[0]

    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

# DQN Agent
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 0.99   # exploration rate
        self.epsilon_min = 0.005
        self.epsilon_decay = 0.995
        self.model = DQNModel(state_size, action_size)
        self.update_target_model()

    def update_target_model(self):
        print('[BEGIN update_target_model]')
        self.model.update_target_model()
        print('[END update_target_model]')

    def remember(self, state, action, reward, next_state, done, pPrint=False):
        print('[BEGIN remember]')
        if pPrint:

            print(f'[state]: {state}')
            print(f'[action]: {action}')
            print(f'[reward]: {reward}')
            print(f'[next_state]: {next_state}')
            print(f'[done]: {done}')
        state = np.eye(self.state_size)[state] if isinstance(state, int) else state
        next_state = np.eye(self.state_size)[next_state] if isinstance(next_state, int) else next_state
        self.memory.append((state, action, reward, next_state, done))
        print('[END remember]')

    ### policy epsilon-greedy
    def act(self, state):
        print('[BEGIN act (policy)]')
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        print('[END act]')
        return np.argmax(act_values)

    def replay(self, batch_size):
      print('[BEGIN replay]')
      minibatch = random.sample(self.memory, min(len(self.memory), batch_size))
      for state, action, reward, next_state, done in minibatch:
          target = self.model.predict(state)

          if done:
              target[action] = reward
          else:
              # Double DQN Logic:
              # 1. Use online network to select the best action in the next state
              best_action = np.argmax(self.model.predict(next_state))
              # 2. Use target network to estimate the Q-value of that best action
              target[action] = reward + self.gamma * self.model.target_model.predict(next_state)[0][best_action]

          # Update the model for the current batch
          self.model.model.fit(np.array([state]), np.array([target]), epochs=1, verbose=0)

      if self.epsilon > self.epsilon_min:
          self.epsilon *= self.epsilon_decay
      print('[END replay]')




    def replay(self, batch_size):
      print('[BEGIN replay]')
      minibatch = random.sample(self.memory, min(len(self.memory), batch_size))
      for state, action, reward, next_state, done in minibatch:
          target = self.model.predict(state)  # Predict Q-values for the current state

          if done:
              target[action] = reward  # If done, target is just the reward
          else:
              # Double DQN Logic:
              # 1. Use online network to select the best action in the next state
              best_action = np.argmax(self.model.predict(next_state))
              # 2. Use target network to estimate the Q-value of that best action
              next_state_reshaped = next_state.reshape(1, -1)  # Reshape next_state for target_model
              target[action] = reward + self.gamma * self.model.target_model.predict(next_state_reshaped, verbose=0)[0][best_action]

          # Update the model for the current batch
          state_reshaped = state.reshape(1, -1)  # Reshape state before fitting
          self.model.model.fit(state_reshaped, np.array([target]), epochs=1, verbose=0)

      if self.epsilon > self.epsilon_min:
          self.epsilon *= self.epsilon_decay
      print('[END replay]')


    def train(self, episodes, batch_size):
        print('[BEGIN train]')
        scores = []
        for e in range(episodes):
            print('')
            print(f'----------------------------------------- ')
            print(f'Episode {e} ')
            state = env.reset()
            state = state[0] if isinstance(state, tuple) else state  # Handle tuple output if it's possible in newer gym versions
            done = False
            score = 0

            iter=0
            while not done and iter <120:
                print(f'iter: {iter}')
                iter+=1

                action = self.act(state)
                next_state, reward, terminated, truncated, info = env.step(action)
                done = terminated or truncated
                reward = reward if not done else -10  # Penalize for not solving the environment

                self.remember(state, action, reward, next_state, done)
                state = next_state
                score += reward

                print(f'[epidosde] {e} - [iter] {iter}  - [ score ] : {score}')
                if done :
                    scores.append(score)
                    print(f"Episode: {e}/{episodes}, Score: {score}, Epsilon: {self.epsilon:.2}")
                    if e % 10 == 0:
                        self.update_target_model()
                    break

                # Call replay every 7 iterations instead of every action
                if len(self.memory) > batch_size and iter % 7 == 0:
                    self.replay(batch_size)
        print('[END train]')

        return scores

In [ ]:
# Training the agent
agent = DQNAgent(state_size, action_size)
scores = agent.train(500, batch_size=32)

# Plotting the results
plt.plot(scores)
plt.title('Training Scores over Episodes')
plt.xlabel('Episode')
plt.ylabel('Score')
plt.show()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


[BEGIN update_target_model]
[END update_target_model]
[BEGIN train]

----------------------------------------- 
Episode 0 
iter: 0
[BEGIN act (policy)]
[BEGIN remember]
[END remember]
[epidosde] 0 - [iter] 1  - [ score ] : -10
iter: 1
[BEGIN act (policy)]
[BEGIN remember]
[END remember]
[epidosde] 0 - [iter] 2  - [ score ] : -11
iter: 2
[BEGIN act (policy)]
[BEGIN remember]
[END remember]
[epidosde] 0 - [iter] 3  - [ score ] : -21
iter: 3
[BEGIN act (policy)]
[BEGIN remember]
[END remember]
[epidosde] 0 - [iter] 4  - [ score ] : -22
iter: 4
[BEGIN act (policy)]
[BEGIN remember]
[END remember]
[epidosde] 0 - [iter] 5  - [ score ] : -23
iter: 5
[BEGIN act (policy)]
[BEGIN remember]
[END remember]
[epidosde] 0 - [iter] 6  - [ score ] : -33
iter: 6
[BEGIN act (policy)]
[BEGIN remember]
[END remember]
[epidosde] 0 - [iter] 7  - [ score ] : -34
iter: 7
[BEGIN act (policy)]
[BEGIN remember]
[END remember]
[epidosde] 0 - [iter] 8  - [ score ] : -35
iter: 8
[BEGIN act (policy)]
[BEGIN remember]

KeyboardInterrupt: 